# Creating Custom Routers

This notebook provides a comprehensive guide to creating your own custom routers in LLMRouter.

## Overview

LLMRouter uses a modular architecture where all routers inherit from `MetaRouter`. This design allows you to:

1. **Create simple rule-based routers** (no training required)
2. **Build ML-based routers** with custom training logic
3. **Implement API-based routers** that use external services

## Architecture

```
MetaRouter (Base Class)
    ├── route_single(query)  # Route one query
    ├── route_batch(batch)   # Route multiple queries
    ├── save_router(path)    # Save model state
    └── load_router(path)    # Load model state

BaseTrainer (For trainable routers)
    ├── train()              # Training loop
    └── loss_func()          # Loss computation
```

## 1. Environment Setup

In [ ]:
# For Google Colab
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/ulab-uiuc/LLMRouter.git
    %cd LLMRouter
    !pip install -e .
    !pip install pyyaml scikit-learn

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
from llmrouter.utils import setup_environment
setup_environment()

import json
import yaml
import copy
import random
import numpy as np
import torch
import torch.nn as nn
from typing import Any, Dict, List, Optional
from abc import ABC, abstractmethod

print("Environment ready!")

## 2. Understanding the MetaRouter Base Class

All routers must inherit from `MetaRouter` and implement two abstract methods:

| Method | Description | Input | Output |
|--------|-------------|-------|--------|
| `route_single(query)` | Route a single query | `dict` with "query" key | `dict` with "model_name" key |
| `route_batch(batch)` | Route multiple queries | `list` of query dicts | `list` of result dicts |

In [ ]:
# Let's examine the MetaRouter interface
from llmrouter.models.meta_router import MetaRouter

print("MetaRouter Abstract Methods:")
print("=" * 50)
print("1. route_single(query: Dict) -> Dict")
print("   - Routes a single query to a model")
print("   - Must return dict with 'model_name' key")
print()
print("2. route_batch(batch: List) -> List[Dict]")
print("   - Routes multiple queries")
print("   - Can include API calls for execution")
print()
print("MetaRouter provides:")
print("- Automatic YAML config loading")
print("- Data loading via DataLoader")
print("- save_router() / load_router() utilities")

## 3. Example 1: Simple Rule-Based Router

Let's create a router that selects models based on query length.

**Logic**:
- Short queries (< 50 chars) → Small, fast model
- Medium queries (50-200 chars) → Medium model  
- Long queries (> 200 chars) → Large, capable model

In [ ]:
from llmrouter.models.meta_router import MetaRouter

class QueryLengthRouter(MetaRouter):
    """
    A simple router that selects models based on query length.
    
    No training required - pure rule-based routing.
    """
    
    def __init__(self, yaml_path: str = None, thresholds: tuple = (50, 200)):
        """
        Args:
            yaml_path: Path to YAML config (optional)
            thresholds: (short_threshold, long_threshold) for categorizing queries
        """
        # Use dummy model since no neural network is needed
        dummy_model = nn.Identity()
        super().__init__(model=dummy_model, yaml_path=yaml_path)
        
        self.short_threshold = thresholds[0]
        self.long_threshold = thresholds[1]
        
        # Define model mapping (can be overridden via config)
        self.model_mapping = {
            "short": None,   # Will be set from llm_data
            "medium": None,
            "long": None
        }
        
        # Auto-assign models based on size if llm_data is available
        if hasattr(self, 'llm_data') and self.llm_data:
            self._assign_models_by_size()
        
        print(f"QueryLengthRouter initialized!")
        print(f"  Short (<{self.short_threshold} chars): {self.model_mapping['short']}")
        print(f"  Medium: {self.model_mapping['medium']}")
        print(f"  Long (>{self.long_threshold} chars): {self.model_mapping['long']}")
    
    def _assign_models_by_size(self):
        """Automatically assign models based on their size."""
        def parse_size(size_str):
            try:
                size_str = str(size_str).upper().strip()
                if size_str.endswith('B'):
                    return float(size_str[:-1])
                return float(size_str)
            except:
                return 0.0
        
        # Sort models by size
        sorted_models = sorted(
            self.llm_data.items(),
            key=lambda x: parse_size(x[1].get('size', '0'))
        )
        
        if len(sorted_models) >= 3:
            self.model_mapping['short'] = sorted_models[0][0]
            self.model_mapping['medium'] = sorted_models[len(sorted_models)//2][0]
            self.model_mapping['long'] = sorted_models[-1][0]
        elif len(sorted_models) == 2:
            self.model_mapping['short'] = sorted_models[0][0]
            self.model_mapping['medium'] = sorted_models[0][0]
            self.model_mapping['long'] = sorted_models[1][0]
        elif len(sorted_models) == 1:
            self.model_mapping['short'] = sorted_models[0][0]
            self.model_mapping['medium'] = sorted_models[0][0]
            self.model_mapping['long'] = sorted_models[0][0]
    
    def _categorize_query(self, query_text: str) -> str:
        """Categorize query by length."""
        length = len(query_text)
        if length < self.short_threshold:
            return "short"
        elif length > self.long_threshold:
            return "long"
        else:
            return "medium"
    
    def route_single(self, query: Dict[str, Any]) -> Dict[str, Any]:
        """
        Route a single query based on its length.
        
        Args:
            query: Dict with 'query' key containing the text
            
        Returns:
            Dict with original query data plus 'model_name'
        """
        query_text = query.get("query", "")
        category = self._categorize_query(query_text)
        model_name = self.model_mapping[category]
        
        result = copy.copy(query)
        result["model_name"] = model_name
        result["query_category"] = category
        result["query_length"] = len(query_text)
        
        return result
    
    def route_batch(self, batch: Optional[List] = None, task_name: str = None) -> List[Dict]:
        """
        Route a batch of queries.
        
        Args:
            batch: List of query dicts, or None to use test data
            task_name: Optional task name for formatting
            
        Returns:
            List of results with model assignments
        """
        if batch is None:
            if hasattr(self, 'query_data_test'):
                batch = self.query_data_test
            else:
                return []
        
        results = []
        for query in batch:
            if isinstance(query, dict):
                result = self.route_single(query)
            else:
                result = self.route_single({"query": str(query)})
            results.append(result)
        
        return results

In [ ]:
# Create a config file for the custom router
custom_config = {
    "data_path": {
        "llm_data": "data/example_data/llm_candidates/default_llm.json",
        "query_data_test": "data/example_data/query_data/default_query_test.jsonl"
    }
}

CONFIG_PATH = "configs/model_config_train/custom_router_temp.yaml"
os.makedirs(os.path.dirname(CONFIG_PATH), exist_ok=True)

with open(CONFIG_PATH, 'w') as f:
    yaml.dump(custom_config, f)

print("Config saved!")

In [ ]:
# Initialize and test the QueryLengthRouter
router = QueryLengthRouter(yaml_path=CONFIG_PATH)

# Test queries of different lengths
test_queries = [
    {"query": "Hi there!"},  # Short
    {"query": "What is the capital of France and what is its population? Also tell me about its history."},  # Medium
    {"query": "Please provide a comprehensive analysis of the economic, political, and social factors that contributed to the Industrial Revolution in 18th century Britain, including the role of technological innovations, colonial trade, agricultural changes, and the emergence of new social classes. Compare this transformation with similar industrialization processes in other countries."},  # Long
]

print("\nRouting Results:")
print("=" * 70)

for i, query in enumerate(test_queries, 1):
    result = router.route_single(query)
    print(f"\n{i}. Query ({result['query_length']} chars): {query['query'][:50]}...")
    print(f"   Category: {result['query_category']}")
    print(f"   Routed to: {result['model_name']}")

## 4. Example 2: Keyword-Based Router

A router that matches queries to models based on keyword patterns.

**Use Case**: Route math questions to a math-specialized model, code questions to a coding model, etc.

In [ ]:
import re

class KeywordRouter(MetaRouter):
    """
    Routes queries based on keyword matching.
    
    Maps keyword patterns to specific models.
    """
    
    def __init__(self, yaml_path: str = None, keyword_rules: dict = None):
        dummy_model = nn.Identity()
        super().__init__(model=dummy_model, yaml_path=yaml_path)
        
        # Default keyword rules (can be overridden)
        self.keyword_rules = keyword_rules or {
            "math": {
                "patterns": [r"\bcalculate\b", r"\bsolve\b", r"\bequation\b", 
                            r"\bmath\b", r"\d+\s*[+\-*/]\s*\d+", r"\bderivative\b",
                            r"\bintegral\b", r"\bprove\b"],
                "model": None  # Will be assigned
            },
            "code": {
                "patterns": [r"\bcode\b", r"\bprogram\b", r"\bfunction\b",
                            r"\bpython\b", r"\bjavascript\b", r"\bdebug\b",
                            r"\balgorithm\b", r"\bAPI\b"],
                "model": None
            },
            "general": {
                "patterns": [],  # Fallback
                "model": None
            }
        }
        
        # Compile regex patterns
        for category in self.keyword_rules:
            patterns = self.keyword_rules[category]["patterns"]
            self.keyword_rules[category]["compiled"] = [
                re.compile(p, re.IGNORECASE) for p in patterns
            ]
        
        # Assign models from llm_data if available
        if hasattr(self, 'llm_data') and self.llm_data:
            model_names = list(self.llm_data.keys())
            # Simple assignment - you can customize this
            for i, category in enumerate(self.keyword_rules.keys()):
                self.keyword_rules[category]["model"] = model_names[i % len(model_names)]
        
        print("KeywordRouter initialized!")
        for cat, info in self.keyword_rules.items():
            print(f"  {cat}: {info['model']}")
    
    def _match_category(self, query_text: str) -> str:
        """Match query to a category based on keywords."""
        for category, info in self.keyword_rules.items():
            if category == "general":
                continue
            for pattern in info.get("compiled", []):
                if pattern.search(query_text):
                    return category
        return "general"
    
    def route_single(self, query: Dict[str, Any]) -> Dict[str, Any]:
        query_text = query.get("query", "")
        category = self._match_category(query_text)
        model_name = self.keyword_rules[category]["model"]
        
        result = copy.copy(query)
        result["model_name"] = model_name
        result["matched_category"] = category
        return result
    
    def route_batch(self, batch: Optional[List] = None, task_name: str = None) -> List[Dict]:
        if batch is None:
            batch = getattr(self, 'query_data_test', [])
        
        return [self.route_single(q if isinstance(q, dict) else {"query": str(q)}) 
                for q in batch]

In [ ]:
# Test the KeywordRouter
keyword_router = KeywordRouter(yaml_path=CONFIG_PATH)

test_queries = [
    {"query": "Calculate the integral of x^2 from 0 to 5"},
    {"query": "Write a Python function to sort a list"},
    {"query": "What is the capital of Japan?"},
    {"query": "Debug this JavaScript code for me"},
    {"query": "Solve the equation 2x + 5 = 15"},
]

print("\nKeyword Routing Results:")
print("=" * 70)

for query in test_queries:
    result = keyword_router.route_single(query)
    print(f"Query: {query['query'][:50]}...")
    print(f"  Category: {result['matched_category']} -> {result['model_name']}")
    print()

## 5. Example 3: Trainable Custom Router

Now let's create a router that requires training. We'll build a simple logistic regression router.

**Architecture**:
- Extract features from query text
- Train a classifier to predict the best model
- Use BaseTrainer for training logic

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from llmrouter.models.base_trainer import BaseTrainer
import pickle

class TfidfRouter(MetaRouter):
    """
    A trainable router using TF-IDF features and Logistic Regression.
    
    This demonstrates how to create a custom ML-based router.
    """
    
    def __init__(self, yaml_path: str):
        dummy_model = nn.Identity()
        super().__init__(model=dummy_model, yaml_path=yaml_path)
        
        # Initialize TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            stop_words='english',
            ngram_range=(1, 2)
        )
        
        # Initialize classifier
        self.classifier = LogisticRegression(
            max_iter=1000,
            multi_class='multinomial'
        )
        
        # Prepare training data from routing_data_train
        if hasattr(self, 'routing_data_train') and self.routing_data_train is not None:
            # Get best model for each query
            best_routes = self.routing_data_train.loc[
                self.routing_data_train.groupby("query")["performance"].idxmax()
            ].reset_index(drop=True)
            
            self.train_queries = best_routes["query"].tolist()
            self.train_labels = best_routes["model_name"].tolist()
            print(f"Prepared {len(self.train_queries)} training samples")
        
        self.is_trained = False
        print("TfidfRouter initialized!")
    
    def route_single(self, query: Dict[str, Any]) -> Dict[str, Any]:
        if not self.is_trained:
            raise RuntimeError("Router not trained! Call trainer.train() first.")
        
        query_text = query.get("query", "")
        
        # Transform query to TF-IDF features
        features = self.vectorizer.transform([query_text])
        
        # Predict model
        model_name = self.classifier.predict(features)[0]
        probabilities = self.classifier.predict_proba(features)[0]
        confidence = max(probabilities)
        
        result = copy.copy(query)
        result["model_name"] = model_name
        result["confidence"] = float(confidence)
        return result
    
    def route_batch(self, batch: Optional[List] = None, task_name: str = None) -> List[Dict]:
        if batch is None:
            batch = getattr(self, 'query_data_test', [])
        
        return [self.route_single(q if isinstance(q, dict) else {"query": str(q)}) 
                for q in batch]
    
    def save_model(self, path: str):
        """Save trained model to disk."""
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'wb') as f:
            pickle.dump({
                'vectorizer': self.vectorizer,
                'classifier': self.classifier,
                'is_trained': self.is_trained
            }, f)
        print(f"Model saved to: {path}")
    
    def load_model(self, path: str):
        """Load trained model from disk."""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        self.vectorizer = data['vectorizer']
        self.classifier = data['classifier']
        self.is_trained = data['is_trained']
        print(f"Model loaded from: {path}")

In [ ]:
class TfidfRouterTrainer(BaseTrainer):
    """
    Trainer for TfidfRouter.
    
    Handles the training loop for the TF-IDF + LogisticRegression router.
    """
    
    def __init__(self, router: TfidfRouter, save_path: str = None):
        super().__init__(router=router, optimizer=None, device="cpu")
        self.save_path = save_path or "models/tfidf_router/model.pkl"
    
    def train(self, dataloader=None):
        """Train the TF-IDF router."""
        print("Training TfidfRouter...")
        print("=" * 50)
        
        # Get training data from router
        queries = self.router.train_queries
        labels = self.router.train_labels
        
        print(f"Training samples: {len(queries)}")
        print(f"Unique models: {len(set(labels))}")
        
        # Fit TF-IDF vectorizer
        print("\n1. Fitting TF-IDF vectorizer...")
        X = self.router.vectorizer.fit_transform(queries)
        print(f"   Feature matrix shape: {X.shape}")
        
        # Train classifier
        print("\n2. Training classifier...")
        self.router.classifier.fit(X, labels)
        
        # Evaluate on training data
        train_accuracy = self.router.classifier.score(X, labels)
        print(f"   Training accuracy: {train_accuracy:.4f}")
        
        self.router.is_trained = True
        
        # Save model
        print(f"\n3. Saving model...")
        self.router.save_model(self.save_path)
        
        print("\nTraining complete!")
        return {"train_accuracy": train_accuracy}

In [ ]:
# Create config with training data
tfidf_config = {
    "data_path": {
        "llm_data": "data/example_data/llm_candidates/default_llm.json",
        "query_data_test": "data/example_data/query_data/default_query_test.jsonl",
        "routing_data_train": "data/example_data/routing_data/default_routing_train_data.jsonl",
        "routing_data_test": "data/example_data/routing_data/default_routing_test_data.jsonl"
    },
    "model_path": {
        "save_model_path": "models/tfidf_router/model.pkl"
    }
}

TFIDF_CONFIG_PATH = "configs/model_config_train/tfidf_router_temp.yaml"
with open(TFIDF_CONFIG_PATH, 'w') as f:
    yaml.dump(tfidf_config, f)

print("TF-IDF Router config saved!")

In [ ]:
# Initialize and train the TfidfRouter
tfidf_router = TfidfRouter(yaml_path=TFIDF_CONFIG_PATH)

# Create trainer and train
trainer = TfidfRouterTrainer(
    router=tfidf_router,
    save_path="models/tfidf_router/model.pkl"
)

metrics = trainer.train()

In [ ]:
# Test the trained router
test_queries = [
    {"query": "What is machine learning?"},
    {"query": "Solve the quadratic equation x^2 - 5x + 6 = 0"},
    {"query": "Write a function to reverse a string in Python"},
    {"query": "Explain the theory of relativity"},
]

print("\nTF-IDF Router Results:")
print("=" * 70)

for query in test_queries:
    result = tfidf_router.route_single(query)
    print(f"Query: {query['query'][:50]}...")
    print(f"  Model: {result['model_name']}")
    print(f"  Confidence: {result['confidence']:.4f}")
    print()

## 6. Example 4: Ensemble Router

Combine multiple routing strategies for better decisions.

In [ ]:
from collections import Counter

class EnsembleRouter(MetaRouter):
    """
    Combines multiple routers using voting.
    
    Each sub-router votes for a model, and the ensemble
    selects the model with the most votes.
    """
    
    def __init__(self, routers: List[MetaRouter], weights: List[float] = None):
        """
        Args:
            routers: List of router instances to combine
            weights: Optional weights for each router's vote
        """
        dummy_model = nn.Identity()
        super().__init__(model=dummy_model, yaml_path=None)
        
        self.routers = routers
        self.weights = weights or [1.0] * len(routers)
        
        print(f"EnsembleRouter initialized with {len(routers)} sub-routers")
    
    def route_single(self, query: Dict[str, Any]) -> Dict[str, Any]:
        # Collect votes from all routers
        votes = []
        for router, weight in zip(self.routers, self.weights):
            try:
                result = router.route_single(query)
                model_name = result.get("model_name")
                if model_name:
                    votes.extend([model_name] * int(weight * 10))
            except Exception as e:
                print(f"Router {type(router).__name__} failed: {e}")
        
        # Count votes and select winner
        if votes:
            vote_counts = Counter(votes)
            winner = vote_counts.most_common(1)[0][0]
        else:
            winner = "unknown"
        
        result = copy.copy(query)
        result["model_name"] = winner
        result["vote_distribution"] = dict(Counter(votes))
        return result
    
    def route_batch(self, batch: Optional[List] = None, task_name: str = None) -> List[Dict]:
        if batch is None:
            return []
        return [self.route_single(q if isinstance(q, dict) else {"query": str(q)}) 
                for q in batch]

In [ ]:
# Create ensemble with QueryLengthRouter and KeywordRouter
ensemble = EnsembleRouter(
    routers=[router, keyword_router],  # Using previously created routers
    weights=[1.0, 1.5]  # Give more weight to keyword router
)

test_queries = [
    {"query": "Calculate 2 + 2"},
    {"query": "Write a comprehensive essay about the impact of artificial intelligence on modern society, including economic, social, and ethical considerations."},
]

print("\nEnsemble Router Results:")
print("=" * 70)

for query in test_queries:
    result = ensemble.route_single(query)
    print(f"Query: {query['query'][:60]}...")
    print(f"  Winner: {result['model_name']}")
    print(f"  Vote distribution: {result['vote_distribution']}")
    print()

## 7. Best Practices

### Router Design Guidelines

1. **Always inherit from MetaRouter**
   ```python
   class MyRouter(MetaRouter):
       def __init__(self, yaml_path: str):
           dummy_model = nn.Identity()  # If no neural network needed
           super().__init__(model=dummy_model, yaml_path=yaml_path)
   ```

2. **Implement both abstract methods**
   - `route_single()` - For single query routing
   - `route_batch()` - For batch processing (can call route_single internally)

3. **Return consistent output format**
   ```python
   result = copy.copy(query)  # Preserve input
   result["model_name"] = selected_model
   return result
   ```

4. **Use YAML config for flexibility**
   - Data paths
   - Hyperparameters
   - Model paths

5. **Separate training logic into Trainer class**
   - Inherit from `BaseTrainer`
   - Implement `train()` method
   - Keep router focused on inference

In [ ]:
# Template for creating your own router
ROUTER_TEMPLATE = '''
from typing import Any, Dict, List, Optional
import copy
import torch.nn as nn
from llmrouter.models.meta_router import MetaRouter
from llmrouter.models.base_trainer import BaseTrainer


class MyCustomRouter(MetaRouter):
    """
    Description of your router.
    """
    
    def __init__(self, yaml_path: str):
        # Use dummy model if no neural network needed
        dummy_model = nn.Identity()
        super().__init__(model=dummy_model, yaml_path=yaml_path)
        
        # Initialize your router-specific components
        # self.my_model = ...
        
        print("MyCustomRouter initialized!")
    
    def route_single(self, query: Dict[str, Any]) -> Dict[str, Any]:
        """
        Route a single query.
        
        Args:
            query: Dict with 'query' key
            
        Returns:
            Dict with 'model_name' key added
        """
        query_text = query.get("query", "")
        
        # YOUR ROUTING LOGIC HERE
        model_name = "your_selected_model"
        
        result = copy.copy(query)
        result["model_name"] = model_name
        return result
    
    def route_batch(self, batch: Optional[List] = None, 
                    task_name: str = None) -> List[Dict]:
        """
        Route a batch of queries.
        """
        if batch is None:
            batch = getattr(self, 'query_data_test', [])
        
        return [self.route_single(q if isinstance(q, dict) else {"query": str(q)}) 
                for q in batch]


class MyCustomRouterTrainer(BaseTrainer):
    """
    Trainer for MyCustomRouter (if training is needed).
    """
    
    def __init__(self, router: MyCustomRouter, **kwargs):
        super().__init__(router=router, optimizer=None, device="cpu")
    
    def train(self, dataloader=None):
        """
        Training loop.
        """
        print("Training MyCustomRouter...")
        
        # YOUR TRAINING LOGIC HERE
        
        print("Training complete!")
        return {"status": "success"}
'''

print(ROUTER_TEMPLATE)

## 8. File-Based Inference

Load queries from a file and save results.

In [ ]:
import json

# Load queries from a JSONL file
def load_queries_from_file(file_path):
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return queries

# Save results to a JSONL file
def save_results_to_file(results, output_path):
    """Save routing results to a JSONL file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
    print(f"Results saved to: {output_path}")

# Example: Load from default query file
QUERY_FILE = "data/example_data/query_data/default_query_test.jsonl"
OUTPUT_FILE = "outputs/custom_router_results.jsonl"

if os.path.exists(QUERY_FILE):
    # Load queries
    file_queries = load_queries_from_file(QUERY_FILE)
    print(f"Loaded {len(file_queries)} queries from: {QUERY_FILE}")
    
    # Route using our custom QueryLengthRouter
    file_results = router.route_batch(batch=file_queries[:10])
    print(f"Routed {len(file_results)} queries")
    
    # Save results
    save_results_to_file(file_results, OUTPUT_FILE)
    
    # Show sample results
    print(f"\nSample results:")
    for i, result in enumerate(file_results[:3], 1):
        print(f"  {i}. {result.get('query', '')[:40]}...")
        print(f"     Category: {result.get('query_category', 'N/A')}")
        print(f"     Model: {result['model_name']}")
else:
    print(f"Query file not found: {QUERY_FILE}")
    print("Create a JSONL file with format: {\"query\": \"Your question\"}")

## Summary

This notebook demonstrated how to create custom routers:

| Example | Type | Training | Use Case |
|---------|------|----------|----------|
| QueryLengthRouter | Rule-based | No | Simple length-based routing |
| KeywordRouter | Rule-based | No | Domain-specific routing |
| TfidfRouter | ML-based | Yes | Text classification routing |
| EnsembleRouter | Hybrid | No | Combining multiple strategies |

**Key Takeaways**:
1. Inherit from `MetaRouter` for consistent interface
2. Implement `route_single()` and `route_batch()` methods
3. Use `BaseTrainer` for trainable routers
4. YAML config provides flexibility
5. Always return dict with `model_name` key

**Next Steps**:
- Experiment with different routing strategies
- Combine with evaluation using `Evaluator`
- Deploy your custom router in production